In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('D:/Machin Language Projects/air quality/air_quality_health_impact_data.csv')
pd.set_option('max_colwidth', 100)
df = df.drop(columns=['RecordID'])
df.head(5)

,AQI,PM10,PM2_5,NO2,SO2,O3,Temperature,Humidity,WindSpeed,RespiratoryCases,CardiovascularCases,HospitalAdmissions,HealthImpactScore,HealthImpactClass
0,187.270059,295.853039,13.038560,6.639263,66.161150,54.624280,5.150335,84.424344,6.137755,7,5,1,97.244041,0.0
1,475.357153,246.254703,9.984497,16.318326,90.499523,169.621728,1.543378,46.851415,4.521422,10,2,0,100.000000,0.0
2,365.996971,84.443191,23.111340,96.317811,17.875850,9.006794,1.169483,17.806977,11.157384,13,3,0,100.000000,0.0
3,299.329242,21.020609,14.273403,81.234403,48.323616,93.161033,21.925276,99.473373,15.302500,8,8,1,100.000000,0.0
4,78.009320,16.987667,152.111623,121.235461,90.866167,241.795138,9.217517,24.906837,14.534733,9,0,1,95.182643,0.0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
# Özellikler ve hedef değişkeni belirleme
X = df.drop(columns=['HealthImpactClass'])
y = df['HealthImpactClass']

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

## 1) Boosting

Boosting, zayıf öğrenicileri (weak learners) güçlü bir tahminciye dönüştürmeyi amaçlayan bir makine öğrenimi topluluk (ensemble) tekniğidir. Boosting, her bir zayıf öğrenicinin hatalarını düzelterek modelin doğruluğunu artırır.

Boosting'in prensipleri şunlardır:

- Adım Adım Öğrenme: Boosting, zayıf öğrenicileri ardışık olarak eğitir. Her yeni model, önceki modellerin hatalarını düzelterek daha iyi tahmin yapmaya çalışır.
- Ağırlıklandırma: Her bir örneğin hata oranına göre ağırlıklandırılır. Hatalı sınıflandırılmış örnekler daha yüksek ağırlık alır, böylece sonraki model bu örnekleri daha iyi öğrenir.
- Topluluk Kararı: Nihai tahmin, tüm zayıf öğrenicilerin tahminlerinin ağırlıklı toplamı veya çoğunluk kararı ile belirlenir.

### 1.1 Gradient Boosting

Gradient Boosting, zayıf modellerin ardışık olarak eğitilmesiyle güçlü bir model oluşturur. Her bir model, önceki modellerin hatalarını düzelterek öğrenir.

In [5]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting modelini oluşturma
gb_model = GradientBoostingClassifier(random_state=58)
gb_model.fit(X_train, y_train)

# Tahmin yapma
y_pred = gb_model.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[948   5   3   7   3]
 [  6 103   0   2   0]
 [ 10   3  42   0   1]
 [  6   1   0   7   0]
 [ 15   1   0   0   0]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97       966
         1.0       0.91      0.93      0.92       111
         2.0       0.93      0.75      0.83        56
         3.0       0.44      0.50      0.47        14
         4.0       0.00      0.00      0.00        16

    accuracy                           0.95      1163
   macro avg       0.65      0.63      0.64      1163
weighted avg       0.94      0.95      0.94      1163



### 1.2 AdaBoost

AdaBoost, zayıf öğreniciler (genellikle decision tree'ler) kullanarak ardışık olarak modeller oluşturur ve her adımda hataları düzelterek güçlü bir model oluşturur.

In [6]:
from sklearn.ensemble import AdaBoostClassifier

# AdaBoost modelini oluşturma
ab_model = AdaBoostClassifier(random_state=58)
ab_model.fit(X_train, y_train)

# Tahmin yapma
y_pred = ab_model.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[939   1   5   0  21]
 [  8 100   2   0   1]
 [ 10   2  39   0   5]
 [  6   1   1   6   0]
 [ 15   0   0   0   1]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.97      0.97       966
         1.0       0.96      0.90      0.93       111
         2.0       0.83      0.70      0.76        56
         3.0       1.00      0.43      0.60        14
         4.0       0.04      0.06      0.05        16

    accuracy                           0.93      1163
   macro avg       0.76      0.61      0.66      1163
weighted avg       0.94      0.93      0.94      1163



### 1.3 XGBoost

XGBoost, performans ve hız açısından oldukça güçlü bir boosting algoritmasıdır. Aynı zamanda eksik verilerle başa çıkma yeteneği de vardır.

In [7]:
import xgboost as xgb

# XGBoost modelini oluşturma
xgb_model = xgb.XGBClassifier(random_state=58)
xgb_model.fit(X_train, y_train)

# Tahmin yapma
y_pred = xgb_model.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[964   1   1   0   0]
 [  7 104   0   0   0]
 [ 10   3  43   0   0]
 [  6   1   0   7   0]
 [ 16   0   0   0   0]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       966
         1.0       0.95      0.94      0.95       111
         2.0       0.98      0.77      0.86        56
         3.0       1.00      0.50      0.67        14
         4.0       0.00      0.00      0.00        16

    accuracy                           0.96      1163
   macro avg       0.78      0.64      0.69      1163
weighted avg       0.95      0.96      0.95      1163



## 2) Majority Voting

Majority voting ensemble, birden fazla makine öğrenimi modelinin tahminlerini birleştirerek nihai sınıflandırma kararını veren bir ensemble yöntemidir. Bu yöntemde, her bir modelin verdiği sınıf tahminleri alınır ve en çok oy alan sınıf, nihai tahmin olarak seçilir. Bu yaklaşım, farklı modellerin güçlü yönlerini birleştirerek daha doğru ve sağlam bir tahmin yapmayı amaçlar.

Çalışma Prensibi
Farklı Modellerin Eğitimi: Farklı türde makine öğrenimi modelleri (örneğin, decision tree, k-nearest neighbors, logistic regression) aynı veri seti üzerinde eğitilir.
Tahminlerin Alınması: Test verileri üzerinde her bir modelin tahminleri yapılır.
Oyların Birleştirilmesi: Her bir veri noktası için modellerin yaptığı tahminler alınır ve en çok oyu alan sınıf, nihai tahmin olarak belirlenir.

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Veri setini yükleyin ve bağımlı ve bağımsız değişkenleri ayır
X = df.drop(columns=['HealthImpactClass'])
y = df['HealthImpactClass']

# Veri setini eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

# Modelleri tanımla
log_clf = LogisticRegression(random_state=58, max_iter=10000)
knn_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf = DecisionTreeClassifier(random_state=58)

# Majority Voting Ensemble oluştur
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='hard'  # 'hard' majority voting, 'soft' probability averaging
)

# Ansambl modelini eğit
voting_clf.fit(X_train, y_train)

# Tahmin yapın
y_pred = voting_clf.predict(X_test)

# Sonuçları değerlendir
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[964   1   1   0   0]
 [  9 102   0   0   0]
 [ 12  11  33   0   0]
 [  6   1   3   4   0]
 [ 16   0   0   0   0]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       966
         1.0       0.89      0.92      0.90       111
         2.0       0.89      0.59      0.71        56
         3.0       1.00      0.29      0.44        14
         4.0       0.00      0.00      0.00        16

    accuracy                           0.95      1163
   macro avg       0.75      0.56      0.61      1163
weighted avg       0.93      0.95      0.94      1163



## 3) Bagging (Bootstrap Aggregating)

Bagging yöntemi, modelin genel performansını artırmak amacıyla birden fazla modelin aynı veri setinden rastgele seçilmiş alt örnekler üzerinde eğitildiği bir ensemble yöntemidir. Bagging, overfitting riskini azaltarak modelin genelleme yeteneğini artırır. Bu yöntem, özellikle yüksek varyansa sahip modellerde etkili sonuçlar verir.
Bu yönteme örnek olarak Random Forest’ı ayrı bir .ipynb dosyasında kullandım. Random Forest da bir bagging yöntemidir.

## 4) Stacking

Stacking, birden fazla modelin tahminlerini bir araya getirerek, nihai tahmini yapmak için bu tahminleri ikinci bir modelle (meta-learner) öğrenen bir ensemble yöntemidir. Stacking, temel öğrenicilerin (base learners) zayıf yanlarını telafi ederek genel model performansını artırmayı amaçlar.

Stacking'in Adımları
Temel Öğrenicilerin Eğitimi (Base Learners):

Farklı algoritmalardan bir dizi temel model (örneğin, lojistik regresyon, SVM, karar ağaçları) eğitilir.
Bu modeller veri setinin farklı özelliklerini yakalamaya çalışır.
Meta Öğrenicinin Eğitimi (Meta-Learner):

Temel öğrenicilerin tahminleri bir araya getirilir ve bu tahminler, yeni bir model (meta-learner) ile birleştirilir.
Meta-learner, temel öğrenicilerin tahminlerine dayanarak nihai tahmini yapar.

Stacking, çok sınıflı veri setlerinde farklı model türlerinin güçlü yönlerini bir araya getirerek daha iyi bir genel performans sağlayabilir. Özellikle bu veri setinde, çeşitli hava kalitesi ve sağlık etkisi ölçümlerine dayanan çok sayıda sınıf bulunduğundan, stacking gibi bir ensemble yöntemi, sınıflandırma performansını artırmada etkili olabilir.

In [9]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib  # Import joblib for saving the model

# Veri setini yükleme
# df = pd.read_csv('path_to_dataset.csv')

# Özellikler ve hedef değişkeni belirleme
X = df.drop(columns=['HealthImpactClass'])
y = df['HealthImpactClass']

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

# Temel öğrenicileri tanımlama
estimators = [
    ('lr', LogisticRegression(max_iter=1000, random_state=58)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier(random_state=58))
]

# StackingClassifier'ı tanımlama
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=58),
    cv=5
)

# Modeli eğitme
stacking_clf.fit(X_train, y_train)

# Tahmin yapma
y_pred = stacking_clf.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Modeli pkl dosyasına kaydetme
model_filename = 'stacking_model.pkl'
joblib.dump(stacking_clf, model_filename)
print(f"Model has been saved to {model_filename}")


Confusion Matrix:
[[964   1   1   0   0]
 [  9 102   0   0   0]
 [ 10   4  42   0   0]
 [  6   1   0   7   0]
 [ 16   0   0   0   0]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       966
         1.0       0.94      0.92      0.93       111
         2.0       0.98      0.75      0.85        56
         3.0       1.00      0.50      0.67        14
         4.0       0.00      0.00      0.00        16

    accuracy                           0.96      1163
   macro avg       0.78      0.63      0.68      1163
weighted avg       0.95      0.96      0.95      1163

Model has been saved to stacking_model.pkl


In [10]:
X_train

,AQI,PM10,PM2_5,NO2,SO2,O3,Temperature,Humidity,WindSpeed,RespiratoryCases,CardiovascularCases,HospitalAdmissions,HealthImpactScore
1826,20.975612,186.994330,166.287124,25.023232,23.810563,289.637143,27.244114,88.407586,8.258407,6,7,1,89.130981
4105,331.912237,36.755743,144.606886,92.895229,53.317759,113.354413,5.748297,17.671456,4.323641,9,3,0,100.000000
498,487.197404,242.639833,69.798987,103.431194,30.291707,293.535944,14.792186,99.756776,13.108582,16,3,4,100.000000
874,12.693178,4.837394,131.118790,145.470703,94.978710,282.354123,17.381739,67.895914,10.603093,11,7,3,76.647627
2987,252.397799,37.963593,141.740858,81.201155,56.605416,202.773220,28.868166,61.510565,6.120312,14,6,1,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5210,473.843089,12.601730,114.675358,5.326337,45.759913,40.140420,32.882702,48.581990,2.151754,11,7,1,100.000000
3828,420.454355,6.488098,113.240965,74.545341,49.103362,64.147984,1.844158,77.755383,3.547843,19,7,0,100.000000
2965,350.584931,292.107640,36.153984,167.770317,69.537853,32.846913,-6.544956,13.474710,2.212371,12,6,5,100.000000
3982,112.159279,79.363886,52.918207,72.211940,32.469286,106.022767,19.174126,56.485073,13.930092,6,4,3,69.959538


In [11]:
import joblib
import pandas as pd

# Step 1: Load the saved model (replace with your actual .pkl file path)
model = joblib.load('stacking_model.pkl')  # Make sure to use the correct path to your model

# Step 2: Create the input data (must have the same columns as used during training)
data = {
      # Add the missing column with a placeholder value (0 or None)
    'AQI': [120],  # Example input values (replace with your actual input)
    'PM10': [40],
    'PM2_5': [30],
    'NO2': [15],
    'SO2': [5],
    'O3': [45],
    'Temperature': [22],
    'Humidity': [55],
    'WindSpeed': [5],
    'RespiratoryCases': [200],
    'CardiovascularCases': [150],
    'HospitalAdmissions': [30],
    'HealthImpactScore': [80]
}

# Step 3: Convert input data to DataFrame
input_data = pd.DataFrame(data)

# Step 4: No scaling applied (since it wasn't during training)
input_data_scaled = input_data  # No scaling is done

# Step 5: Make predictions
prediction = model.predict(input_data_scaled)

# Step 6: Output the prediction
print("Predicted Health Impact Class:", prediction)


Predicted Health Impact Class: [2.]


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# Load the dataset (replace 'path_to_dataset.csv' with your actual dataset file)
df = pd.read_csv('D:/Machin Language Projects/air quality/air_quality_health_impact_data.csv')

# Define features and target (assuming 'HealthImpactClass' is the target column)
X = df.drop(columns=['HealthImpactClass','RecordID'])

# Fit the scaler on the features
scaler = StandardScaler()
scaler.fit(X)

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

print("Scaler saved as 'scaler.pkl'")


Scaler saved as 'scaler.pkl'


NameError: name 'X' is not defined